In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
ZIP_to_CBSA = pd.read_excel('CBSA_ZIP_032020.xlsx')

In [3]:
ZIP_to_CBSA.rename(columns = {'ZIP':'zip'}, inplace = True) 

In [35]:
ZIP_to_CBSA.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47415 entries, 0 to 47414
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   CBSA       47415 non-null  int64  
 1   zip        47415 non-null  int64  
 2   RES_RATIO  47415 non-null  float64
 3   BUS_RATIO  47415 non-null  float64
 4   OTH_RATIO  47415 non-null  float64
 5   TOT_RATIO  47415 non-null  float64
dtypes: float64(4), int64(2)
memory usage: 2.2 MB


In [5]:
Hospital_Outpatient = pd.read_excel('MUP_OHP_R19_P04_V10_D17_APC_Provider.xlsx',header=5)

In [6]:
APC_to_CPT_HPCS = pd.read_excel('2020_january_web_addendum_b.12312019.xlsx', header = 2)


In [67]:
APC_to_CPT_HPCS.rename(columns = {'HCPCS Code':'hcpcs_code'}, inplace = True) 
APC_to_CPT_HPCS.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16628 entries, 0 to 16627
Data columns (total 10 columns):
 #   Column                                                                                                        Non-Null Count  Dtype  
---  ------                                                                                                        --------------  -----  
 0   hcpcs_code                                                                                                    16628 non-null  object 
 1   Short Descriptor                                                                                              16628 non-null  object 
 2   SI                                                                                                            16628 non-null  object 
 3   APC                                                                                                           5942 non-null   float64
 4   Relative Weight                                     

In [8]:
phys_other_payments = pd.read_csv('Medicare_Provider_Util_Payment_PUF_CY2017.txt', sep='\t',low_memory=False)

In [9]:
phys_other_payments = phys_other_payments.drop(phys_other_payments.index[0])


In [10]:
phys_other_payments.rename(columns={'nppes_provider_last_org_name':'provider_last_org_name',
                          'nppes_provider_first_name':'provider_first_name',
                          'nppes_provider_mi':'provider_mi',
                          'nppes_credentials':'credentials',
                          'nppes_provider_gender ':'provider_gender',
                          'nppes_entity_code':'entity_code',
                          'nppes_provider_street1':'provider_street1',
                          'nppes_provider_street2':'provider_street2',
                          'nppes_provider_city':'provider_city',
                          'nppes_provider_zip':'zip',
                          'nppes_provider_state':'provider_state',
                          'nppes_provider_country':'country'},inplace=True)


In [11]:
phys_other_payments['zip'] = phys_other_payments['zip'].str[:5]

In [12]:
phys_other_payments = phys_other_payments[phys_other_payments.country == 'US']

In [33]:
phys_other_payments['zip'] = pd.to_numeric(phys_other_payments['zip'])

In [14]:
phys_other_payments['zip'] = phys_other_payments['zip'].apply(lambda x: '{0:0>5}'.format(x))

In [31]:
phys_other_payments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9846847 entries, 1 to 9847443
Data columns (total 26 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   npi                               int64  
 1   provider_last_org_name            object 
 2   provider_first_name               object 
 3   provider_mi                       object 
 4   credentials                       object 
 5   nppes_provider_gender             object 
 6   entity_code                       object 
 7   provider_street1                  object 
 8   provider_street2                  object 
 9   provider_city                     object 
 10  zip                               object 
 11  provider_state                    object 
 12  country                           object 
 13  provider_type                     object 
 14  medicare_participation_indicator  object 
 15  place_of_service                  object 
 16  hcpcs_code                        ob

In [62]:
"""Goal 1 filters for physician/other supplier"""
Goal1_Physician = phys_other_payments[(phys_other_payments['entity_code']=='O') & (phys_other_payments['hcpcs_code']=='43249')]
Goal1_Physician.head()

,npi,provider_last_org_name,provider_first_name,provider_mi,credentials,nppes_provider_gender,entity_code,provider_street1,provider_street2,provider_city,zip,provider_state,country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt
2921,1003015843,"NORTHBAY PHYSICIAN'S SURGERY CENTER, L.L.C.",NaN,NaN,NaN,NaN,O,1006 NUT TREE ROAD,NaN,VACAVILLE,95687,CA,US,Ambulatory Surgical Center,Y,F,43249,Balloon dilation of esophagus using an endoscope,N,20.0,16.0,20.0,814.53450,2408.4,596.400500,447.812500
14114,1003079187,WASC LLC,NaN,NaN,NaN,NaN,O,3373 COMMERCE PARKWAY,SUITE 1,WOOSTER,44691,OH,US,Ambulatory Surgical Center,Y,F,43249,Balloon dilation of esophagus using an endoscope,N,17.0,15.0,17.0,555.31000,1850.0,433.162941,474.892941
27010,1003220187,"CENTRAL OHIO ENDOSCOPY CENTER, LLC",NaN,NaN,NaN,NaN,O,3400 OLENTANGY RIVER RD,NaN,COLUMBUS,43202,OH,US,Ambulatory Surgical Center,Y,F,43249,Balloon dilation of esophagus using an endoscope,N,94.0,79.0,94.0,596.51000,750.5,463.132234,478.148830
41895,1003815242,"DOTHAN SURGERY CENTER, LLC",NaN,NaN,NaN,NaN,O,1450 ROSS CLARK CIR,SUITE 4,DOTHAN,36301,AL,US,Ambulatory Surgical Center,Y,F,43249,Balloon dilation of esophagus using an endoscope,N,446.0,409.0,446.0,509.85778,1300.0,391.675538,477.716614
74448,1003887225,CENTRAL TEXAS ENDOSCOPY CENTER LLC,NaN,NaN,NaN,NaN,O,2206 E. VILLA MARIA,NaN,BRYAN,77802,TX,US,Ambulatory Surgical Center,Y,F,43249,Balloon dilation of esophagus using an endoscope,N,26.0,23.0,26.0,576.83000,1752.0,445.692692,470.542692


In [76]:
Goal1_HO = phys_other_payments[(phys_other_payments['entity_code']=='I')]
Goal1_HO.head()

,npi,provider_last_org_name,provider_first_name,provider_mi,credentials,nppes_provider_gender,entity_code,provider_street1,provider_street2,provider_city,zip,provider_state,country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt
1,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,21502,MD,US,Internal Medicine,Y,F,99217,Hospital observation care discharge,N,100.0,96.0,100.0,73.398800,325.780000,56.827200,57.492400
2,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,21502,MD,US,Internal Medicine,Y,F,99218,Hospital observation care typically 30 minutes,N,26.0,25.0,26.0,100.080000,449.000000,78.460000,79.306154
3,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,21502,MD,US,Internal Medicine,Y,F,99219,Hospital observation care typically 50 minutes,N,52.0,51.0,52.0,136.380000,614.000000,102.807692,103.895385
4,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,21502,MD,US,Internal Medicine,Y,F,99220,Hospital observation care typically 70 minutes...,N,59.0,59.0,59.0,190.363729,755.932203,141.293559,142.865763
5,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,21502,MD,US,Internal Medicine,Y,F,99221,"Initial hospital inpatient care, typically 30 ...",N,16.0,16.0,16.0,101.680000,462.812500,79.710000,80.750000
